In [9]:
import re
from collections import Counter

import codecs
import numpy as np
import gensim

import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from tqdm.notebook import tqdm

In [10]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [11]:
with codecs.open('plut.txt', encoding='UTF8', mode='r') as f:    
    raw_docs = f.readlines()

# docs = [re.sub(r'[^А-я ]', '', doc) for doc in raw_docs]
docs = ' '.join(raw_docs)

In [12]:
def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])
    
    return sequence, char_to_idx, idx_to_char

sequence, char_to_idx, idx_to_char = text_to_seq(docs)

[' ', 'о', 'е', 'а', 'и', 'н', 'т', 'с', 'л', 'в', 'р', 'к', 'д', 'м', 'у', 'п', 'ы', ',', 'я', 'ь', 'з', 'б', 'г', 'ч', 'й', '.', 'х', 'ж', 'ш', '—', 'ю', '\n', 'щ', 'ц', 'э', 'Н', 'П', 'В', 'О', 'К', '!', 'Г', 'А', 'С', '-', ';', 'М', '?', 'ф', 'И', 'Т', ':', 'З', 'Д', 'Е', 'Э', 'Б', 'Р', 'Я', 'У', 'Ч', 'Л', '«', '»', 'ъ', '1', 'Ж', '…', 'Ш', 'Х', '2', '9', '8', '3', '0', '(', ')', '[', ']', '4', '5', 'Ы', 'Ф', 'Ц', 'Ю', '6', 'Й', 'Щ', 'Ь', '_', '7', '„', '“', 'e', 's', '–', 'i', '+', 'I', 'n', 'a', 'o', 'r', 'X', 'c', 'u', 'v', 'm', 'l', 'ё', '°', 'g', 'h', 't', '*', '/', 'w', 'b', "'", 'Ъ', 'D', 'R', 'd', 'H', 'f', 'S', 'K', 'O', 'T', 'p', 'P', 'C', 'y']


In [13]:
SEQ_LEN = 256
BATCH_SIZE = 16

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [14]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    return predicted_text

In [15]:
class TextRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [16]:
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

n_epochs = 50000
loss_avg = []

for epoch in (pbar := tqdm(range(n_epochs))):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)
    
    pbar.set_description(f'[Epoch: {epoch + 1:2d}] loss: {loss.item():.3f}')

  0%|          | 0/50000 [00:00<?, ?it/s]

Loss: 3.066613554954529
 подни отоли полосто стостони косто но в онол водо вони сно сто вень но посто но потой столи поло в болол восков полом черел ете отоли постато посталь ваве токат в полак кене востосто по ноза помони по
Loss: 2.5554444885253904
 тора не поредерени столы сторо бо сторо востольни в бо кодолье тольно столеми столой содать на сколости в сто отсто полали больи в домсти поредне сторо породить ны полити порости поромали сторал на и 
Loss: 2.3478259372711183
 попотомо на полого и стали и стала ворутно породи и в долять и на в подного со востали и на полодельно вожно отраники соверенно пополомом на стором в просте в сладали востелько потором половали на под
Loss: 2.2132659482955934
 баланов и полодовали в полодить с долкам достров стали от долта приставший на полодели на стровало в стали на полодельно на простых и от ветров состровали в нестом по на канных но стровал от на было в
Loss: 2.1136361169815063
 постельно с в порого с поровенных и по от на полодно подновались от

KeyboardInterrupt: 

In [18]:
model.eval()

print(evaluate(
    model, 
    char_to_idx, 
    idx_to_char, 
    temp=0.3, 
    prediction_len=1000, 
    start_text='кто на втором'
    )
)

кто на втором на севере вода и берега было бы в поляны в долину с обратно и поднялись на стойбищами и подняли свое странные полости и подножия подняли вода своей землянки, и на обратно на северо-привленных положение дальше с ними на севере и собаки поднялись в сторону и покрыто на север. Так как поднял за которой не стали стояли в несколько стояли на востоке и собаками и собаками пришлось высокой стены подняли поляны и последнего получить на обратно по глубины подняли по воде было воскликнул в образовали на севере которого воды по поляну и моря и после половина половину и последний в поляны и воды и совершенно поднялись по сторону, которые от нас по своей подняли на север, на развитица и уже на полосы по странными поднимались в землянки и стороны и подобные против с остальных положением в сторону с образом в делать и подножия вода в которой не обрыва и представляла море с оставили бы не было воды с только в своей стороны и вода высоких промежутки и подняли своим в сторону и на север, п